In [1]:
import pandas as pd

In [2]:
from data_quality import cargue_datasets_c
casos, hurto_tp, mede_victimas, compar = cargue_datasets_c()

In [3]:
from data_quality import casos_c
casos=casos_c(casos)

In [4]:
from data_quality import hurto_tp_c
hurto_tp= hurto_tp_c(hurto_tp)

In [6]:
#df = pd.merge(casos, on=["Comuna","Año"], how="inner")
#df = pd.merge(df, hurto_tp, on=["Comuna","Año"], how="inner")
#df = pd.merge(df, mede_victimas, on=["Comuna","Año"], how="inner")
#df = pd.merge(df, compar, on=["Comuna","Año"], how="inner")
#df

TypeError: merge() missing 1 required positional argument: 'right'